In [1]:
import os
import sys
import json

sys.path.append("src")

from semi_supervised_mlp import LatentMLPClassifier, load_latents_and_labels
from train_latent_classifier import train_latent_classifier, evaluate_model 

In [2]:
latents_path = "latents.npy"
labels_path = "cluster_labels.npy"
save_path = "mlp_classifier.pth"
log_path = "mlp_training_log.json"

In [3]:
from sklearn.model_selection import train_test_split
import torch

# Load full dataset
latents, labels = load_latents_and_labels(latents_path, labels_path)

# Step 1: Split into Train+Val and Test (85/15)
X_temp, X_test, y_temp, y_test = train_test_split(latents, labels, test_size=0.15, random_state=42, stratify=labels)

# Step 2: Split Train+Val into Train and Val (approx 70/15 overall)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42, stratify=y_temp)
# 0.1765 ≈ 15% / 85% to get 15% overall val

# Optional: Print shapes
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Step 3: Train the model
train_latent_classifier(
    latents_path=latents_path,
    labels_path=labels_path,
    val_data=(X_val, y_val),
    save_path="mlp_classifier.pth",
    log_path="mlp_training_log.json",
    epochs=30,
    batch_size=64,
    lr=1e-3
)

# Step 4: Evaluate on test set
evaluate_model("mlp_classifier.pth", X_test, y_test)


Train: torch.Size([5304, 16384]), Val: torch.Size([1138, 16384]), Test: torch.Size([1137, 16384])
Epoch 1/30, Train Loss: 0.02093017
   🔎 Val Loss: 0.00000000
Epoch 2/30, Train Loss: 0.00000523
   🔎 Val Loss: 0.00000000
Epoch 3/30, Train Loss: 0.00000096
   🔎 Val Loss: 0.00000000
Epoch 4/30, Train Loss: 0.00000005
   🔎 Val Loss: 0.00000000
Epoch 5/30, Train Loss: 0.00000082
   🔎 Val Loss: 0.00000000
Epoch 6/30, Train Loss: 0.00000030
   🔎 Val Loss: 0.00000000
Epoch 7/30, Train Loss: 0.00000019
   🔎 Val Loss: 0.00000000
Epoch 8/30, Train Loss: 0.00000023
   🔎 Val Loss: 0.00000000
Epoch 9/30, Train Loss: 0.00000007
   🔎 Val Loss: 0.00000000
Epoch 10/30, Train Loss: 0.00000093
   🔎 Val Loss: 0.00000000
Epoch 11/30, Train Loss: 0.00000078
   🔎 Val Loss: 0.00000000
Epoch 12/30, Train Loss: 0.00000138
   🔎 Val Loss: 0.00000000
Epoch 13/30, Train Loss: 0.00000003
   🔎 Val Loss: 0.00000000
Epoch 14/30, Train Loss: 0.00000031
   🔎 Val Loss: 0.00000000
Epoch 15/30, Train Loss: 0.00000496
   🔎 Va

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL semi_supervised_mlp.LatentMLPClassifier was not an allowed global by default. Please use `torch.serialization.add_safe_globals([LatentMLPClassifier])` or the `torch.serialization.safe_globals([LatentMLPClassifier])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about

In [4]:
with open(log_path, 'r') as f:
    log = json.load(f)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(log["loss_curve"], label="Loss")
plt.axhline(log["best_loss"], color='red', linestyle='--', label=f"Best Loss: {log['best_loss']:.4f}")
plt.title("Training Loss (MLP on Latents)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [6]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
latents, labels = load_latents_and_labels(latents_path, labels_path)

model = LatentMLPClassifier(input_dim=latents.shape[1])
model.load_state_dict(torch.load(save_path))
model.to(device)
model.eval()

with torch.no_grad():
    preds = model(latents.to(device)).cpu().numpy()
    preds_bin = (preds >= 0.5).astype(int)
    y_true = labels.numpy()

In [ ]:
print("✅ Evaluation Metrics:")
print(f"🔢 Accuracy: {accuracy_score(y_true, preds_bin):.4f}")
print("\n📊 Classification Report:")
print(classification_report(y_true, preds_bin))
print("🧮 Confusion Matrix:")
print(confusion_matrix(y_true, preds_bin))